In [1]:
# import libraries
import pandas as pd
import datetime as dt

In [2]:
# All patients (large and small) with cancer stage > 0
data = pd.read_csv("../data/data.csv")

In [3]:
# Number of patients IB2 and above
len(data[data["combined_cancer_stage_exact"] >= 6]["patient_id"])

902

In [7]:
# For patients IB2 and above:
ib2_above_w_treatment = data[(data["combined_cancer_stage_exact"] >= 6) &
                             (data["treatment"] == 1)]

# Number of IB and above patients who do not have treatment (RT) start date
print(len(ib2_above_w_treatment[ib2_above_w_treatment["actual_treat_start_date"].isnull()]["patient_id"]))
print(sorted(ib2_above_w_treatment[ib2_above_w_treatment["actual_treat_start_date"].isnull()]["patient_id"]))
# Export these patient ids
ib2_above_w_treatment[ib2_above_w_treatment["actual_treat_start_date"].isnull()]["patient_id"].to_excel("../data/data_for_surbhi_to_check/no_treat_start_date_patient_ids.xlsx", index = False)
# Of these, 3 received surgery
print(len(ib2_above_w_treatment[(ib2_above_w_treatment["actual_treat_start_date"].isnull()) & (ib2_above_w_treatment["surgery"] == 1)]))

7
['242.0', '5233.0', '5251.0', '5429.0', '5677.0', '5945.0', '6107.0']
3


In [8]:
# Assess time to treatment
ib2_above_w_treatment["path_to_treatment_difference"] = (pd.to_datetime(ib2_above_w_treatment["actual_treat_start_date"]) - pd.to_datetime(ib2_above_w_treatment["pathology_date"])).dt.days

ib2_above_w_treatment["path_to_treatment_difference"].describe()

# Export information for Surbhi
#ib2_above[["patient_id", "pathology_date", "actual_treat_start_date", "path_to_treatment_difference"]].to_excel("../data/data_for_surbhi_to_check/time_to_treatment.xlsx", index = False)

# Check patients who have negative delay
#ib2_above[ib2_above["path_to_treatment_difference"] < 0][["patient_id", "pathology_date", "actual_treat_start_date", "path_to_treatment_difference"]].to_excel("../data/data_for_surbhi_to_check/time_to_treatment_negative.xlsx", index = False)



<ipython-input-8-a7040d3a8128>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ib2_above_w_treatment["path_to_treatment_difference"] = (pd.to_datetime(ib2_above_w_treatment["actual_treat_start_date"]) - pd.to_datetime(ib2_above_w_treatment["pathology_date"])).dt.days


count     829.000000
mean      114.422195
std       143.690700
min      -260.000000
25%        54.000000
50%        81.000000
75%       124.000000
max      1978.000000
Name: path_to_treatment_difference, dtype: float64

In [9]:
# 13 with NA for treatment delay time
ib2_above_w_treatment[ib2_above_w_treatment["path_to_treatment_difference"].isnull()][["patient_id", "actual_treat_start_date", "pathology_date", "path_to_treatment_difference"]]


,patient_id,actual_treat_start_date,pathology_date,path_to_treatment_difference
107,242.0,NaN,2016-04-05,NaN
159,351.0,2016-11-21,NaN,NaN
163,358.0,2016-12-13,NaN,NaN
220,468.0,2017-06-20,NaN,NaN
294,675.0,2018-07-05,NaN,NaN
456,5233.0,NaN,2015-09-23,NaN
460,5251.0,NaN,2016-12-29,NaN
525,5429.0,NaN,2017-06-27,NaN
609,5677.0,NaN,2018-01-31,NaN
697,5945.0,NaN,2018-11-01,NaN


In [12]:
# 13 rows with negative treatment delay time after updates with Rebecca
ib2_above_w_treatment[ib2_above_w_treatment["path_to_treatment_difference"] < 0][["patient_id", "pathology_date", "actual_treat_start_date", "treat_start_date"]]
# these patients were originally in Rebecca's list with exception of: 169 

,patient_id,pathology_date,actual_treat_start_date,treat_start_date
17,37.0,2015-03-27,2015-03-20,2015-03-20 00:00:00
41,83.0,2015-06-20,2015-06-10,2015-06-10 00:00:00
102,236.0,2016-08-26,2016-05-19,2016-05-19 00:00:00
459,5250.0,2017-02-06,2017-02-03,2017-02-03 00:00:00
563,5530.0,2017-09-12,2017-09-10,2017-09-10 00:00:00
583,5608.0,2018-03-22,2018-02-08,2018-02-08 00:00:00
618,5704.0,2018-04-20,2017-08-16,2017-08-16 00:00:00
622,5715.0,2018-05-31,2018-05-24,2018-05-24 00:00:00
804,6261.0,2019-10-06,2019-01-19,2019-01-19 00:00:00
810,6273.0,2019-10-10,2019-09-12,2019-09-12 00:00:00


In [13]:
# exclude patients with missing treatment delay or negative treatment delay
treatment_delay_data = ib2_above_w_treatment[ib2_above_w_treatment["path_to_treatment_difference"] >= 0]

treatment_delay_data.shape

(816, 123)

In [14]:
# Save delay data
treatment_delay_data.to_excel("../data/time_to_treatment_data.xlsx", index = False)


### Add new survival endpoint (24 months) to data

In [2]:
delay_data = pd.read_excel("../data/time_to_treatment_data.xlsx")
delay_data.head()

,patient_id,enroll_age,age_cat,hiv_status,marital,distance,cancer_screening,combined_cancer_stage,combined_cancer_stage_exact,cr_result,...,actual_treat_start_date,init_cancer_stage,init_cancer_stage_exact,final_cancer_stage,final_cancer_stage_exact,chemo_only,primary_surgery_rt,path_to_treatment_difference,curative_palliative_status,vital_status24
0,1.0,50.0,2.0,1.0,2.0,59.2,1.0,4,12,90.0,...,2015-02-03,4.0,12.0,3.0,10.0,0,0,75,Definitive,0
1,3.0,40.0,2.0,1.0,1.0,310.0,0.0,3,10,597.0,...,2015-02-04,3.0,10.0,NaN,NaN,0,0,181,Definitive,1
2,5.0,48.0,2.0,1.0,3.0,0.0,0.0,3,10,50.0,...,2015-02-04,3.0,10.0,3.0,10.0,0,0,8,Definitive,0
3,9.0,65.0,3.0,0.0,3.0,59.2,0.0,2,8,60.0,...,2015-02-17,2.0,8.0,NaN,NaN,0,0,127,Curative,1
4,11.0,51.0,2.0,1.0,3.0,0.0,1.0,3,9,117.0,...,2015-01-22,NaN,NaN,3.0,9.0,0,0,164,Curative,1


In [3]:
delay_data.columns.values

array(['patient_id', 'enroll_age', 'age_cat', 'hiv_status', 'marital',
       'distance', 'cancer_screening', 'combined_cancer_stage',
       'combined_cancer_stage_exact', 'cr_result', 'hb_result',
       'neut_result', 'wbc_result', 'alb_result',
       'init_performance_status', 'chemo', 'total_chemo_received',
       'ebrt_curr_dose', 'brachy_curr_dose', 'eqd2',
       'ebrt_boost_curr_dose', 'treat_duration', 'treat_response',
       'vital_status', 'death_date', 'treat_start_date',
       'vaginal_hemorrhage', 'vaginal_discharge', 'pelvic_pain',
       'cd4_final', 'vl_final', 'nnrti', 'nrti', 'pi', 'on_arv',
       'lamivudine', 'abacavir', 'amprenavir', 'azt', 'atazanavir',
       'atazanavir_boosted', 'cobicistat', 'stavudine', 'zalcitabine',
       'didanosine', 'delavirdine', 'darunavir', 'darunavir_boosted',
       'dolutegravir', 'efavirenz', 'etravirine', 'elvitegravir',
       'fosamprenavir', 'fosamprenavir_boosted', 'emtricitabine', 'gazt',
       'indinavir', 'lopinav

In [ ]:
# print(delay_data["combined_cancer_stage"].unique())
# delay_data["combined_cancer_stage_exact"].unique()

delay_data.rename(columns={"combined_cancer_stage":"init_combined_cancer_stage", 
                           "combined_cancer_stage_exact":"init_combined_cancer_stage_exact"}, inplace=True)

delay_data["final_combined_cancer_stage_exact"] = delay_data["final_cancer_stage_exact"].fillna(delay_data["init_cancer_stage_exact"])
delay_data["final_combined_cancer_stage"] = delay_data["final_cancer_stage"].fillna(delay_data["init_cancer_stage"])

In [5]:
print(delay_data[delay_data["time_alive_treat"] >= 731].shape)
print(delay_data[delay_data["time_alive_treat"] < 731].shape)

(397, 123)
(419, 123)


In [6]:
delay_data.loc[delay_data["time_alive_treat"] >= 731, "vital_status24"] = 1
delay_data.loc[delay_data["time_alive_treat"] < 731, "vital_status24"] = 0

In [6]:
delay_data.to_excel("../data/time_to_treatment_data.xlsx", index=False)